### Basic Inheritance
All Python classes are subclasses of the special built-in class named `object`. Its attributes and methods are:

In [2]:
for i in dir(object):
    print(i, end=' ')

__class__ __delattr__ __dir__ __doc__ __eq__ __format__ __ge__ __getattribute__ __gt__ __hash__ __init__ __init_subclass__ __le__ __lt__ __ne__ __new__ __reduce__ __reduce_ex__ __repr__ __setattr__ __sizeof__ __str__ __subclasshook__ 

To inherit from class Parent, we write
```py
class Child(Parent):
    pass
```

In [3]:
class Contact:
    all_contacts = []

    def __init__(self, name, email):
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)
        
# For Supplier, we don't have explicitly set __init__
# Supplier simply inherits Contact's __init__
class Supplier(Contact):
    def order(self, order):
        print(
            "If this were a real system we would send "
            f"'{order}' order to '{self.name}'"
        )
        
# Overriding __init__
# But this
class Friend(Contact): 
    def __init__(self, name, email, phone):         
        super().__init__(name, email)
        self.phone = phone

### Multiple Inheritance
A **mixin** is a superclass that is not intended to exist on its own, but is meant to be inherited by some other class to provide extra functionality. For example, MailSender in below example is a mixin.

In [5]:
class MailSender: 
    def send_mail(self, message): 
        print("Sending mail to " + self.email)

# The __init__ method is called from left to right
# Which means Contact first and then MailSender
class EmailableContact(Contact, MailSender): 
    def __init__(self, name, email):
        super().__init__(name, email)

emailableContact = EmailableContact('John Doe', 'john.doe@email.com')

Consider the case when C inherits from A and B where A and B have different signaturs,

In [6]:
class A(object):
    def __init__(self, a, b):
        print('Init {} with arguments {}'.format(self.__class__.__name__, (a, b)))

class B(object):
    def __init__(self, q):
        print('Init {} with arguments {}'.format(self.__class__.__name__, (q)))

class C(A, B):
    def __init__(self):
        # Unbound functions, so pass in self explicitly
        A.__init__(self, 1, 2)
        B.__init__(self, 3)

Or use the super function and do the same in a super ugly fashion,

In [13]:
class A(object):
    def __init__(self, a=None, b=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.a = a
        self.b = b
        print('Init {} with arguments {}'.format(self.__class__.__name__, (a, b)))

class B(object):
    def __init__(self, q=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.q = q
        print('Init {} with arguments {}'.format(self.__class__.__name__, (q)))

class C(A, B):
    def __init__(self):
        super().__init__(a=1, b=2, q=3)
        
c = C()
print(c.q)

Init C with arguments 3
Init C with arguments (1, 2)
3


Now consider that A and B have a common attribute r,

In [15]:
class A(object):
    def __init__(self, a, r):
        self.a = a
        self.r = r
        print('Init {} with arguments {}'.format(self.__class__.__name__, (a, r)))

class B(object):
    def __init__(self, b, r):
        self.b = b
        self.r = r
        print('Init {} with arguments {}'.format(self.__class__.__name__, (b, r)))

class C(A, B):
    def __init__(self):
        # Unbound functions, so pass in self explicitly
        A.__init__(self, 1, 2)
        B.__init__(self, 3, 4)
        
c = C()
print(c.r)

Init C with arguments (1, 2)
Init C with arguments (3, 4)
4


### Method Resolution Order
MRO determines the order in which parent classe's methods are called. Consider the following class hierarchy:
```
    A   B
    | \/|
    | /\|
    |/ \|
    X   Y
    \  /
     \/
     O
     
O(X,Y)
X(A,B)
Y(B,A)
```
The MRO of class X is \[XAB\] which means that if a method is called for X, it is first searched in X, then in A and at last in B.

Let's take a more complex example,
```
O
D(O)
E(O)
F(O)
B(D,E)
C(D,F)
A(B,C)
```

Here the term $L[B]$ is called linearization of B which is effectively the MRO for B. We'll start from the top.  
$$L[O] = O$$
As per C3 algorithm,
$$L[D] = D + merge(L[O], O)$$
$$L[D] = D + merge(O, O)$$
$$L[D] = DO$$
Similary, 
$$L[E] = EO$$
$$L[F] = FO$$
Now,
$$L[B] = B + merge(L[D], L[E], DE)$$
$$L[B] = B + merge(DO, EO, DE)$$
To calculate merge, we take the tail of the first list which is tail of DO which is O. We see that O is not present in tail of the remaining lists (EO and DE). So we write,
$$L[B] = B + D + merge(O, EO, E)$$
Which is basically removing D from inside the merge function. Note that both O and E inside the merge function are head and not tail. Now we take O. O is present in tail of EO. So we move on to EO. Head of EO is E. It is not present in tail of remaining list (E). So,
$$L[B] = B + D + E + merge(O, O)$$
$$L[B] = B + D + E + O$$
$$L[B] = BDEO$$
In a similar fashion, 
$$L[C] = CDFO$$
Now we need to calculate, MRO of A. Note that in CDFO C is head, whereas DFO is tail.
$$L[A] = A + merge(L[B], L[C], BC)$$
$$L[A] = A + merge(BDEO, CDFO, BC)$$
$$L[A] = A + B + merge(DEO, CDFO, C)$$
D exists in the tail of CDFO, therefore move to the next list CDFO. Head of CDFO, C doesn't exist in tail of C, so remove it from inside merge.
$$L[A] = A + B + C + merge(DEO, DFO)$$
Now head of DEO, D doesn't exist in tail of DFO, So we remove it. Remember to goto first list inside merge after every removal.
$$L[A] = A + B + C + D + merge(EO, FO)$$
Now head of EO, E doesn't exist in tail of FO
$$L[A] = A + B + C + D + E + merge(O, FO)$$
O exists in tail of FO,
$$L[A] = A + B + C + D + E + F + merge(O, O)$$
$$L[A] = A + B + C + D + E + F + O$$
$$L[A] = ABCDEFO$$